In [289]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter

In [290]:
data = pd.read_csv('movie_bd_v5.csv', parse_dates = ['release_date'])
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
767,tt1161864,37000000,96047633,The Rite,Anthony Hopkins|Colin O'Donoghue|Alice Braga|T...,Mikael HÃ¥fstrÃ¶m,You can only defeat it when you believe.,Seminary student Michael Kovak (Colin O'Donogh...,114,Drama|Thriller|Horror,New Line Cinema|Contrafilm|Mid Atlantic Films|...,2011-01-28,5.7,2011
1610,tt0363547,28000000,102356381,Dawn of the Dead,Sarah Polley|Ving Rhames|Jake Weber|Mekhi Phif...,Zack Snyder,"When the undead rise, civilization will fall.",A group of surviving people take refuge in a s...,101,Fantasy|Horror|Action,New Amsterdam Entertainment|Strike Entertainme...,2004-03-19,6.7,2004
1656,tt0329691,40000000,46546197,Torque,Martin Henderson|Ice Cube|Monet Mazur|Adam Sco...,Joseph Kahn,Circuit of Fire,Biker Cary Ford is framed by an old rival and ...,84,Action,Village Roadshow Pictures|Original Film|Warner...,2004-01-16,4.8,2004
1150,tt0304711,35000000,11560806,The Order,Heath Ledger|Shannyn Sossamon|Benno FÃ¼rmann|M...,Brian Helgeland,Every soul has its price.,"For centuries, a secret Order of priests has e...",102,Drama|Fantasy|Horror|Mystery|Romance,Twentieth Century Fox Film Corporation|N1 Euro...,2003-09-05,4.8,2003
272,tt0362478,30000000,33333531,The Box,Cameron Diaz|James Marsden|Frank Langella|Mich...,Richard Kelly,All you have to do is push the button.,"Norma and Arthur Lewis, a suburban couple with...",115,Thriller|Science Fiction,Media Rights Capital|Lin Pictures|Warner Bros....,2009-09-17,5.3,2009


In [291]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

In [292]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [293]:
answers = {} # создадим словарь для ответов

#
#Ищем столбы с значениями разделенными | и преобразем их в список
for col in data.columns:
    if data[col].dtype == 'object':
        multiple_data = data[col].str.contains('|', regex=False).any()
        print(f'{col} - {multiple_data}')
        #Если есть разделитель, преобразовываем в список
        if multiple_data:
            data[col] = data[col].str.split("|")

# Добавим столбец с прибылью
data['profit'] = data['revenue'] - data['budget'] 

data.sample(5)

imdb_id - False
original_title - False
cast - True
director - True
tagline - False
overview - False
genres - True
production_companies - True


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
541,tt0272020,72000000,27642707,The Last Castle,"[Robert Redford, James Gandolfini, Mark Ruffal...",[Rod Lurie],A Castle Can Only Have One King,A Court Martialed general rallies together 120...,131,"[Action, Drama, Thriller]",[DreamWorks SKG],2001-10-19,7.1,2001,-44357293
1305,tt2034139,5000000,15179302,The Last Exorcism Part II,"[Ashley Bell, Julia Garner, Andrew Sensenig, S...",[Ed Gass-Donnelly],The second coming,As Nell Sweetzer tries to build a new life aft...,88,"[Horror, Thriller]","[StudioCanal, CBS Films]",2013-02-28,4.5,2013,10179302
937,tt0284490,85000000,77741732,Showtime,"[Robert De Niro, Rachael Harris, Eddie Murphy,...",[Tom Dey],Lights. Camera. Aggravation.,A spoof of buddy cop movies where two very dif...,95,"[Action, Comedy]","[Village Roadshow Pictures, NPV Entertainment,...",2002-03-14,5.2,2002,-7258268
159,tt2180411,50000000,212902372,Into the Woods,"[Meryl Streep, Emily Blunt, James Corden, Anna...",[Rob Marshall],Be careful what you wish for...,In a woods filled with magic and fairy tale ch...,125,"[Fantasy, Comedy, Music]","[Walt Disney Pictures, Marc Platt Productions,...",2014-12-25,5.6,2014,162902372
181,tt2024469,50000000,222809600,Non-Stop,"[Liam Neeson, Julianne Moore, Scoot McNairy, M...",[Jaume Collet-Serra],The hijacking was just the beginning.,Bill Marks is a burned-out veteran of the Air ...,106,"[Action, Thriller, Mystery]","[StudioCanal, Silver Pictures, TF1 Films Produ...",2014-01-26,6.8,2014,172809600


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [294]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [295]:
max_budget = data.loc[ data['budget'].idxmax() ]

answers['1']  = max_budget['original_title'] +' (' + max_budget['imdb_id']+')'
answers['1']

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [296]:
#Сортируем по бюджету и выбираем первое значение 
max_budget_v2 = data.sort_values('budget', ascending=False).iloc[0]
print ( max_budget_v2['original_title'] +' (' + max_budget_v2['imdb_id']+')' )

Pirates of the Caribbean: On Stranger Tides (tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [297]:
long_film =  data.loc[ data['runtime'].idxmax() ]

answers['2']  = long_film['original_title'] +' (' + long_film['imdb_id']+')'

print(answers['2'], long_film['runtime'], "минут")

Gods and Generals (tt0279111) 214 минут


# 3. Какой из фильмов самый короткий (в минутах)?





In [298]:
short_film =  data.loc[ data['runtime'].idxmin() ]

answers['3']  = short_film['original_title'] +' (' + short_film['imdb_id']+')'

print(answers['3'], short_film['runtime'], "минут")

Winnie the Pooh (tt1449283) 63 минут


# 4. Какова средняя длительность фильмов?


In [299]:
answers['4'] = data['runtime'].mean();

answers['4'] 

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [300]:
answers['5'] = data['runtime'].median();
answers['5']

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [301]:
max_profit =  data.loc[ data['profit'].idxmax() ]

answers['6']  = max_profit['original_title'] +' (' + max_profit['imdb_id']+')'

print(answers['6'], "прибыль ",max_profit['profit'])

Avatar (tt0499549) прибыль  2544505847


# 7. Какой фильм самый убыточный? 

In [302]:
min_profit = data.loc[ data['profit'].idxmin() ]
answers['7']  = min_profit['original_title'] +' (' + min_profit['imdb_id']+')'

print(answers['7'], "убыток ",-min_profit['profit'])

The Lone Ranger (tt1210819) убыток  165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [303]:
answers['8'] = data[ data['profit'] > 0 ]['imdb_id'].count()
answers['8']

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [304]:
#Выбираем фильмы 2008 года
films2008 = data[ data['release_date'].dt.year == 2008 ]

#Выведем 5 самых кассовых фильмов 2008 года
display( films2008[['original_title', 'release_date', 'revenue'] ].nlargest(5, ['revenue']) )

#Выберем самый кассовый
max_revenue2018 = films2008.loc[ films2008['revenue'].idxmax() ]
answers['9']  = max_revenue2018['original_title'] +' (' + max_revenue2018['imdb_id']+')'
answers['9']

,original_title,release_date,revenue
599,The Dark Knight,2008-07-16,1001921825
603,Indiana Jones and the Kingdom of the Crystal S...,2008-05-21,786636033
606,Kung Fu Panda,2008-06-04,631744560
621,Hancock,2008-07-01,624029371
607,Mamma Mia!,2008-06-30,609841637


'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [305]:
films2012_2014 = data[ (data['release_date'].dt.year >= 2012 ) & (data['release_date'].dt.year <= 2014 ) ]

min_profit2012_2014 = films2012_2014.loc[ films2012_2014['profit'].idxmin() ]
answers['10']  = min_profit2012_2014['original_title'] +' (' + min_profit2012_2014['imdb_id']+')'

answers['10']

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [306]:
#Создадим  список всех жанров без повторений
all_genres = list( set( data['genres'].sum() ) )

#Посчитаем кол-во фильмов по жанрам
g_cnts = []
for genre in all_genres:
    g_cnts.append( data[ data['genres'].str.contains( genre, regex=False, na=False ) ]['imdb_id'].count() )    

# Создадим датафрейм    
genre_films = pd.DataFrame({
    'genre': all_genres,
    'film_counts': g_cnts })
    
#Выведем наиболее выбираемые жанры    
display( genre_films.nlargest(5, ['film_counts']) )
    
    
answers['11'] = genre_films.iloc[ genre_films['film_counts'].idxmax() ]['genre']
answers['11']

,genre,film_counts
17,Drama,782
5,Comedy,683
9,Thriller,596
12,Action,582
7,Adventure,415


'Drama'

ВАРИАНТ 2

In [307]:
#Соберем все жанры в один большрой список
alls_genre = Counter( data['genres'].sum() )

#Выведем 5 наиболее часто встречающихся
display( alls_genre.most_common(5) )

#Посчитаем наиболее частые значения
most_genre = alls_genre.most_common(1)

most_genre[0][0]


[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415)]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [308]:
#Создадим  список всех жанров без повторений
all_genres = list( set( data['genres'].sum() ) )

#Посчитаем кол-во прибыльных фильмов по жанрам
g_profit = []
for genre in all_genres:
    g_profit.append( data[ data['genres'].str.contains( genre, regex=False, na=False ) &
                           data['profit'] > 0 ]['imdb_id'].count() )    

genre_profit_cnt = pd.DataFrame({'profit_cnt': g_profit }, index = all_genres)
      
#Выведем самые частые   
display( genre_profit_cnt.nlargest(5, ['profit_cnt']) )

answers['12'] = genre_profit_cnt['profit_cnt'].idxmax()
answers['12']

,profit_cnt
Drama,375
Comedy,329
Thriller,288
Action,283
Adventure,183


'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [309]:
#Создадим датафрейм со списком всех режисеров
director_films = pd.DataFrame({
    'director': list( set( data['director'].sum() ) ) })

#Добавим колонку с сумарным сбором фильмов, с разделением по режисеру 
director_films['revenue_sum'] = director_films['director'].apply(lambda x: data[ data['director'].str.contains(x,  regex=False, na=False) ]['revenue'].sum() )

#Сделаем индексом режисера
director_films.set_index(['director'], inplace=True)

#Посмотрим самых кассовых режисеров
display( director_films.nlargest(5, ['revenue_sum']) )

#Запишем ответ
answers['13'] = director_films['revenue_sum'].idxmax()
answers['13']

,revenue_sum
director,
Peter Jackson,6490593685
Christopher Nolan,4167548502
David Yates,4154295625
Michael Bay,3886938960
J.J. Abrams,3579169916


'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [310]:
#Создадим датафрейм со списком всех режисеров
director_films = pd.DataFrame({
    'director': list( set( data['director'].sum() ) ) })

#Добавим колонку с сумарным сбором фильмов, с разделением по режисеру 
director_films['action_cnt'] = director_films['director'].apply(lambda x: data[ data['director'].str.contains(x,  regex=False, na=False) & data['genres'].str.contains( "Action", regex=False, na=False ) ]['imdb_id'].count() )

#Сделаем индексом режисера
director_films.set_index(['director'], inplace=True)

display( director_films.nlargest(5, ['action_cnt']) )

answers['14'] = director_films['action_cnt'].idxmax()
answers['14']

,action_cnt
director,
Robert Rodriguez,9
Michael Bay,7
Paul W.S. Anderson,7
Antoine Fuqua,6
Ridley Scott,6


'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [311]:
#Создадим датафрейм со списком всех актеров
cast_revenue2012 = pd.DataFrame({
    'cast': list( set( data['cast'].sum() ) ) })

#Добавим колонку с сумарным сбором фильмов 2012 года, с разделением по актеру 
cast_revenue2012['revenue'] = cast_revenue2012['cast'].apply(lambda x: data[ (data['release_date'].dt.year == 2012) & data['cast'].str.contains(x,  regex=False, na=False) ]['revenue'].sum() )

#Индекс по актеру
cast_revenue2012.set_index(['cast'], inplace=True)

display( cast_revenue2012.nlargest(5, ['revenue']) )

answers['15'] = cast_revenue2012['revenue'].idxmax()
answers['15']

,revenue
cast,
Chris Hemsworth,2027450773
Denis Leary,1629460639
Anne Hathaway,1522851057
Mark Ruffalo,1519557910
Robert Downey Jr.,1519557910


'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [312]:
#Создадим датафрейм со списком всех актеров
casts = pd.DataFrame({
    'cast': list( set( data['cast'].sum() ) ) })

#Добавим колонку с количеством высокобюджетныхфильмов, с разделением по актеру 
casts['most_budget_cnt'] = casts['cast'].apply(lambda x: data[ (data['budget'] >= data['budget'].median()) & data['cast'].str.contains(x,  regex=False, na=False) ]['imdb_id'].count() )

#Меняем индекс на Актера
casts.set_index(['cast'], inplace=True)

display( casts.nlargest(5, ['most_budget_cnt']) )

answers['16'] = casts['most_budget_cnt'].idxmax()
answers['16']

,most_budget_cnt
cast,
Matt Damon,23
Owen Wilson,20
Samuel L. Jackson,20
Johnny Depp,19
Adam Sandler,19


'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [313]:
#Создадим  список всех жанров без повторений
gn_NicCage_cnt = pd.DataFrame({'genre': list( set( data['genres'].sum() ) ) })

#Добавим колонку с колисеством фильмов с Nicolas Cage, в разрезе жанров.
gn_NicCage_cnt['NicCage'] = gn_NicCage_cnt['genre'].apply(lambda x: data[ data['genres'].str.contains( x, regex=False, na=False) & data['cast'].str.contains("Nicolas Cage",  regex=False, na=False) ]['imdb_id'].count() )

#Меняем индекс
gn_NicCage_cnt.set_index(['genre'], inplace=True)

display( gn_NicCage_cnt.nlargest(5, ['NicCage']) )
 
answers['17'] = gn_NicCage_cnt['NicCage'].idxmax()
answers['17']

,NicCage
genre,
Action,17
Thriller,15
Drama,12
Crime,10
Fantasy,8


'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [314]:
#Выберем все фильмы от Paramount Pictures
films_paramount = data[ data['production_companies'].str.contains("Paramount Pictures", regex=False, na=False) ]

#Находим индекс минимального элемента в стобце profit, и по нему получаем всю строку.
min_profit = films_paramount.loc[ films_paramount['profit'].idxmin() ]

display( films_paramount[['original_title','profit']].nsmallest(5, ['profit']) )

answers['18']  = min_profit['original_title'] +' (' + min_profit['imdb_id']+')'
answers['18']

,original_title,profit
925,K-19: The Widowmaker,-64831034
1123,Timeline,-60519261
1722,Next,-51788987
1623,Alfie,-46604061
1631,Twisted,-24805000


'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [315]:
#Группируем данные по годам, извлекаем индекс максимального значения колонки revenue и получаем и него год
answers['19'] = data.resample('Y', on='release_date').sum()['revenue'].idxmax().year
answers['19'] 

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [316]:
#Находим фильмы от Warner Bros
films_WarnerBros = data[ data['production_companies'].str.join("|").str.contains("Warner Bros") ]

#Группируем данные по годам, извлекаем индекс максимального значения колонки profit и получаем и него год
answers['20'] = films_WarnerBros.resample('Y', on='release_date').sum()['profit'].idxmax().year
answers['20'] 

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [317]:
#Групируем данные по месяцам, и подсчитывем кол-во фильмов, 
#из индекса максимального ко-ва фильмов извлекаем название месяца
answers['21'] = data.resample('M', on='release_date')['imdb_id'].count().idxmax().month_name(locale='Russian')
answers['21']

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [318]:
#Выбираем летние фильмы, и считаем кол-во ункальных imdb
answers['22'] = data[ data['release_date'].dt.month.isin([6,7,8])]['imdb_id'].nunique()
answers['22']

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [319]:
#Создадим датафрейм со списком всех режисеров
director_films = pd.DataFrame({
    'director': list( set( data['director'].sum() ) ) })

#Добавим колонку с количеством зимних фильмов, с разделением по режисеру 
director_films['winter_films'] = director_films['director'].apply(lambda x: data[ data['director'].str.contains(x,  regex=False, na=False) & data['release_date'].dt.month.isin([12,1,2]) ]['imdb_id'].count() )

#Сделаем индексом режисера
director_films.set_index(['director'], inplace=True)

display( director_films.nlargest(5, ['winter_films']) )

#Получим идекс максимального значения в колонке winter_films
answers['23'] = director_films['winter_films'].idxmax()
answers['23']

,winter_films
director,
Peter Jackson,7
Clint Eastwood,6
Steven Soderbergh,6
Shawn Levy,4
Martin Scorsese,4


'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [320]:
#Выберем необходимые данные из основной таблицы
title_len = data[['production_companies', 'original_title']].copy()
title_len['title_len'] = title_len['original_title'].apply(len)

#Создадим датафрейм со списком всех компаний
company_films = pd.DataFrame({
    'company': list( set( data['production_companies'].sum() ) ) })

#Добавим колонку с максимальной длиной фильма компании
company_films['max_title_len'] = company_films['company'].apply(lambda x: title_len[ title_len['production_companies'].str.contains(x,  regex=False, na=False) ]['title_len'].max() )

#Выведем 5 компаний с наибольшей длиной названия фильма
display( company_films.nlargest(5, ['max_title_len']) )

answers['24'] = company_films[company_films['max_title_len'] == company_films['max_title_len'].max()  ]['company'].tolist()
answers['24']

,company,max_title_len
22,Twentieth Century Fox Film Corporation,83
1566,Four By Two Productions,83
1025,Walt Disney,62
1300,Walden Media,62
95,"Jim Henson Company, The",59


['Twentieth Century Fox Film Corporation', 'Four By Two Productions']

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [321]:
#Создадим датафрейм со списком всех компаний
company_films = pd.DataFrame({
    'company': list( set( data['production_companies'].sum() ) ) })

#Добавим колонку со средним кол-вом слов в опиcании фильма компании
company_films['overview_word_mean'] = company_films['company'].apply( lambda x: data[ data['production_companies'].str.contains(x,  regex=False, na=False) ]['overview'].str.split().apply(len).mean() )

#Выведем 5 компаний с наибольшим количеством слов в описании фильма
display( company_films.nlargest(5, ['overview_word_mean']) )

#Запишем ответ
answers['25'] = company_films.loc[ company_films['overview_word_mean'].idxmax() ]['company']
answers['25']

,company,overview_word_mean
1422,Midnight Picture Show,175.0
1448,Room 9 Entertainment,161.0
348,98 MPH Productions,159.0
1250,Heineken Branded Entertainment,159.0
494,Brookwell-McNamara Entertainment,156.0


'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [322]:
#Выберем фильмы по 99 процентилю
average99 = data[ data['vote_average'] > data['vote_average'].quantile(.99)   ][['original_title', 'vote_average']]
answers['26'] = average99['original_title'].to_list()
answers['26']

['Inside Out',
 'Room',
 'Interstellar',
 'Guardians of the Galaxy',
 'The Imitation Game',
 'Gone Girl',
 'The Grand Budapest Hotel',
 'Inception',
 'The Dark Knight',
 'The Pianist',
 'The Lord of the Rings: The Return of the King',
 'The Wolf of Wall Street',
 '12 Years a Slave',
 'Memento']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [323]:
#all_cast = list( set( data['cast'].sum() ) )

#Сократим список актеров, т.к. задача на поиск по всему списку актеров содержит 11 597 430 вариантов комбинаций.
find_cast = ['Johnny Depp' , 'Helena Bonham Carter',
'Ben Stiller' , 'Owen Wilson',
'Vin Diesel' , 'Paul Walker',
'Adam Sandler' , 'Kevin James',
'Daniel Radcliffe' , 'Rupert Grint' ]


#Выберим только список актеров фильма, и преобразуем их во множество
cast_film_set = data['cast'].apply( set )

max_films = -1;

#Комбинации из 2 актеров
for casts in itertools.combinations(find_cast, 2):
    s_cast = set(casts)
    #Ищем вхождение множества с комбинацией актеров в список актеров фильма и считаем их кол-во    
    film_cnt = cast_film_set.apply(lambda x: s_cast.issubset(x)).sum()
    #В max_cast записываем комбинацию актеров с максимальным кол-вом фильмов
    if film_cnt > max_films:
        max_films = film_cnt
        max_cast = casts 
        
print( max_cast , max_films, 'фильмов' )

answers[27] = max_cast


('Daniel Radcliffe', 'Rupert Grint') 8 фильмов


# Submission

In [324]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 109.6585494970884,
 '5': 107.0,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': ['Twentieth Century Fox Film Corporation', 'Four By Two Productions'],
 '25': 'Midnight Picture Show',
 '26': ['Inside Out',
  'Room',
  'Interstellar',
  'Guardians of the Galaxy',
  'The Imitation Game',
  'Gone Girl',
  'The Grand Budapest Hotel',
  'Inception',
  'The Dark Knight',
  'The Pianist',
  'The Lord of the Rings: The Return of the King',
  'The Wolf of Wall Street',
  '12 Y

In [325]:
# и убедиться что ни чего не пропустил)
len(answers)

27